In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import string, unicodedata
import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [2]:
from contraction import fix
import inflect
import time

In [3]:
os.getcwd()

'C:\\Users\\sunilchawla\\Desktop\\ass2\\code'

In [4]:
path = "../data_df/"

In [5]:
df_train = pd.read_csv(path+"pos_neg.csv",sep = ",")
df_test = pd.read_csv(path+"pos_neg_test.csv",sep = ",")
df_unsup = pd.read_csv(path+"unsup.csv",sep = ",")

In [6]:
df_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
df_test.drop(['Unnamed: 0'], axis = 1, inplace = True)
df_unsup.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [7]:
df_train.head()

,movieID,review,rate,label
0,0,Story of a man who has unnatural feelings for ...,3,0
1,10000,Airport '77 starts as a brand new luxury 747 p...,4,0
2,10001,This film lacked something I couldn't put my f...,4,0
3,10002,"Sorry everyone,,, I know this is supposed to b...",1,0
4,10003,When I was little my parents took me along to ...,1,0


In [8]:
df_test.head()

,movieID,review,rate,label
0,0,Once again Mr. Costner has dragged out a movie...,2,0
1,10000,This is an example of why the majority of acti...,4,0
2,10001,"First of all I hate those moronic rappers, who...",1,0
3,10002,Not even the Beatles could write songs everyon...,3,0
4,10003,Brass pictures (movies is not a fitting word f...,3,0


In [9]:
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunilchawla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sunilchawla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sunilchawla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
df_train['words'] = np.nan
df_test['words'] = np.nan
df_unsup['words'] = np.nan

In [14]:
df_train['review']=df_train['review'].str.lower()
df_test['review'] =df_test['review'].str.lower()
df_unsup['review'] =df_unsup['review'].str.lower()

In [15]:
stop_word = ["above" ,"below", "up", "down", "over", "all", "few", "most", "no", "not", "too", "very", "won", "again", "nor"]

In [16]:
stop_word

['above',
 'below',
 'up',
 'down',
 'over',
 'all',
 'few',
 'most',
 'no',
 'not',
 'too',
 'very',
 'won',
 'again',
 'nor']

In [17]:
from nltk.corpus import stopwords
from string import punctuation
customStopWords=[]
for x in stopwords.words('english'):
     if x not in stop_word:
            customStopWords.append(x)
customStopWords = set(customStopWords+list(punctuation)+['hmm','hmmm','hmmmm','hmmmmm','hmmmmmm','hmmmmmmm','hmmmmmmmm'])

In [18]:
customStopWords

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'after',
 'against',
 'ain',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'during',
 'each',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'hmm',
 'hmmm',
 'hmmmm',
 'hmmmmm',
 'hmmmmmm',
 'hmmmmmmm',
 'hmmmmmmmm',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'mustn',
 "mustn't",
 'my',
 'my

In [19]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [20]:
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

In [21]:
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [22]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return fix(text)

In [23]:
def normalize(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = denoise_text(text)
    text = replace_contractions(text)
    return text

In [24]:
df_train['review'] = df_train['review'].apply(lambda x: normalize(x))

In [25]:
df_test['review'] = df_test['review'].apply(lambda x: normalize(x))

In [26]:
df_unsup['review'] = df_unsup['review'].apply(lambda x: normalize(x))

In [27]:
df_train['review'] 

0        story of a man who has unnatural feelings for ...
1        airport '77 starts as a brand new luxury 747 p...
2        this film lacked something i could not put my ...
3        sorry everyone,,, i know this is supposed to b...
4        when i was little my parents took me along to ...
5        "it appears that many critics find the idea of...
6        the second attempt by a new york intellectual ...
7        i do not know who to blame, the timid writers ...
8        this film is mediocre at best. angie harmon is...
9        the film is bad. there is no other way to say ...
10       this film is one giant pant load. paul schrade...
11       the plot for descent, if it actually can be ca...
12       plot is not worth discussion even if it hints ...
13       this film is about a male escort getting invol...
14       this movie must be in line for the most boring...
15       a worn-out plot of a man who takes the rap for...
16       i saw this movie at a drive-in in 1959. until .

In [28]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [29]:
def removed_stop_word(text):
    word_sent = word_tokenize(text)
    text = [lmtzr.lemmatize(word) for word in word_sent if word not in customStopWords]
    return text

In [30]:
df_train['words'] = df_train['review'].apply(lambda x: removed_stop_word(x))

In [31]:
df_test['words'] = df_test['review'].apply(lambda x: removed_stop_word(x))

In [32]:
df_unsup['words'] = df_unsup['review'].apply(lambda x: removed_stop_word(x))

In [33]:
df_train['words'] 

0        [story, man, unnatural, feeling, pig, start, o...
1        [airport, '77, start, brand, new, luxury, 747,...
2        [film, lacked, something, could, not, put, fin...
3        [sorry, everyone, know, supposed, ``, art, '',...
4        [little, parent, took, along, theater, see, in...
5        [``, appears, many, critic, find, idea, woody,...
6        [second, attempt, new, york, intellectual, le,...
7        [not, know, blame, timid, writer, clueless, di...
8        [film, mediocre, best, angie, harmon, funny, b...
9        [film, bad, no, way, say, story, weak, outdate...
10       [film, one, giant, pant, load, paul, schrader,...
11       [plot, descent, actually, called, plot, two, n...
12       [plot, not, worth, discussion, even, hint, cor...
13       [film, male, escort, getting, involved, murder...
14       [movie, must, line, most, boring, movie, year,...
15       [worn-out, plot, man, take, rap, woman, murder...
16       [saw, movie, drive-in, 1959., ``, howard, duck.

In [34]:
df_train['words_sent'] = [" ".join(list_sent) for list_sent in df_train['words'].values]

In [35]:
df_test['words_sent'] = [" ".join(list_sent) for list_sent in df_test['words'].values]

In [36]:
df_unsup['words_sent'] = [" ".join(list_sent) for list_sent in df_unsup['words'].values]

In [37]:
t = time.time()
for i in range(25000):
    df_train.at[i,'words_sent'] = re.sub("<br />|``|br br|''|--","",df_train.at[i,'words_sent'])
    df_test.at[i,'words_sent'] = re.sub("<br />|``|br br|''|--","",df_test.at[i,'words_sent'])
elapsed = time.time() - t
print(elapsed)

1.339360237121582


In [38]:
t = time.time()
for i in range(50000):
    df_unsup.at[i,'words_sent'] = re.sub("<br />|``|br br|''|--","",df_unsup.at[i,'words_sent'])
elapsed = time.time() - t
print(elapsed)

1.0319340229034424


In [39]:
df_train.head()

,movieID,review,rate,label,words,words_sent
0,0,story of a man who has unnatural feelings for ...,3,0,"[story, man, unnatural, feeling, pig, start, o...",story man unnatural feeling pig start opening ...
1,10000,airport '77 starts as a brand new luxury 747 p...,4,0,"[airport, '77, start, brand, new, luxury, 747,...",airport '77 start brand new luxury 747 plane l...
2,10001,this film lacked something i could not put my ...,4,0,"[film, lacked, something, could, not, put, fin...",film lacked something could not put finger fir...
3,10002,"sorry everyone,,, i know this is supposed to b...",1,0,"[sorry, everyone, know, supposed, ``, art, '',...",sorry everyone know supposed art film wow ha...
4,10003,when i was little my parents took me along to ...,1,0,"[little, parent, took, along, theater, see, in...",little parent took along theater see interior ...


In [40]:
df_test.head()

,movieID,review,rate,label,words,words_sent
0,0,once again mr. costner has dragged out a movie...,2,0,"[again, mr., costner, dragged, movie, far, lon...",again mr. costner dragged movie far longer nec...
1,10000,this is an example of why the majority of acti...,4,0,"[example, majority, action, film, generic, bor...",example majority action film generic boring re...
2,10001,"first of all i hate those moronic rappers, who...",1,0,"[first, all, hate, moronic, rapper, could'nt, ...",first all hate moronic rapper could'nt act gun...
3,10002,not even the beatles could write songs everyon...,3,0,"[not, even, beatles, could, write, song, every...",not even beatles could write song everyone lik...
4,10003,brass pictures (movies is not a fitting word f...,3,0,"[brass, picture, movie, not, fitting, word, re...",brass picture movie not fitting word really so...


In [41]:
df_train['words_sent']

0        story man unnatural feeling pig start opening ...
1        airport '77 start brand new luxury 747 plane l...
2        film lacked something could not put finger fir...
3        sorry everyone know supposed  art  film wow ha...
4        little parent took along theater see interior ...
5         appears many critic find idea woody allen dra...
6        second attempt new york intellectual le 10 yea...
7        not know blame timid writer clueless director ...
8        film mediocre best angie harmon funny bag hamm...
9        film bad no way say story weak outdated especi...
10       film one giant pant load paul schrader utterly...
11       plot descent actually called plot two notewort...
12       plot not worth discussion even hint corruption...
13       film male escort getting involved murder inves...
14       movie must line most boring movie year not eve...
15       worn-out plot man take rap woman murder case e...
16       saw movie drive-in 1959.  howard duck  conside.

In [42]:
df_train['words_sent']

0        story man unnatural feeling pig start opening ...
1        airport '77 start brand new luxury 747 plane l...
2        film lacked something could not put finger fir...
3        sorry everyone know supposed  art  film wow ha...
4        little parent took along theater see interior ...
5         appears many critic find idea woody allen dra...
6        second attempt new york intellectual le 10 yea...
7        not know blame timid writer clueless director ...
8        film mediocre best angie harmon funny bag hamm...
9        film bad no way say story weak outdated especi...
10       film one giant pant load paul schrader utterly...
11       plot descent actually called plot two notewort...
12       plot not worth discussion even hint corruption...
13       film male escort getting involved murder inves...
14       movie must line most boring movie year not eve...
15       worn-out plot man take rap woman murder case e...
16       saw movie drive-in 1959.  howard duck  conside.

In [43]:
df_train.to_csv("../data_clean/pos_neg.csv")

In [44]:
df_test.to_csv("../data_clean/pos_neg_test.csv")

In [45]:
df_unsup.to_csv("../data_clean/unsup.csv")